#### myae2

In [5]:
from torchvision import datasets
from torchvision.transforms import ToTensor

def order_dataset(default=False):
    set_names = ['mnist', 'fashion-mnist']
    if default:
        idx_set = 0
    else:
        idx_set = int(input(str([f'{idx}: {n}' for idx, n in enumerate(set_names)])))
    sets = [datasets.MNIST, datasets.FashionMNIST]
    data = sets[idx_set](root='/home/secondvoca/sdc_ae_23/data', train=True, download=True, transform=ToTensor())
    return data
data = order_dataset()
data.data.shape

torch.Size([60000, 28, 28])

In [ ]:
class Data_Keeper:
    def prepare(self, less_than=10, batch_size=128, shuffle=True):
        training_data = datasets.MNIST(
            root="data",
            train=True,
            download=True,
            transform=ToTensor(),
        )

        training_data.data = training_data.data[training_data.targets < less_than]
        training_data.targets = training_data.targets[training_data.targets < less_than]

        self.training_data = training_data.data / 255.0
        self.training_targets = training_data.targets

        self.training_data_length = len(training_data.data)

        self.train_dataloader = DataLoader(
            training_data, batch_size=batch_size, shuffle=shuffle
        )

In [ ]:
class Model_builder:
    

In [ ]:
class Manager:
    

    def set_model(self, encoder, decoder):
        self.model = nn.Sequential(
            OrderedDict(
                [
                    ("encoder", encoder),
                    ("decoder", decoder),
                ]
            )
        )
        self.optimizer = torch.optim.Adam(self.model.parameters())

    def get_cuda_device_or_cpu(self):
        if torch.cuda.is_available():
            cuda_count = torch.cuda.device_count()

            no = 0
            mem_available = 0

            for i in range(cuda_count):
                tmp_available = torch.cuda.mem_get_info(i)[0]
                if mem_available < tmp_available:
                    no = i
                    mem_available = tmp_available
            return f"cuda:{no}"
        return "cpu"

    def save_current_model(self, name):
        torch.save(
            self.model,
            f'./models/{datetime.datetime.today().strftime("%Y-%m-%d %H:%M:%S")}_{name}.pt',
        )

    def load_model(self, name):
        self.model = torch.load(f"./models/{name}.pt")

    def run(self, model, dataloader, optimizer, device, calc_loss):
        hist = torch.zeros(len(dataloader))

        for batch, (x, y) in enumerate(dataloader):
            x = x.view([-1, 28 * 28]).to(device)
            y = y.to(device)

            # Compute prediction error
            loss = calc_loss(model, x, y, F, device=device)

            # Backpropagation
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            hist[batch] = loss.item()

        return hist

    def train(self, calc_loss, epochs=5, need_latent_record=False, ):
        try:
            device = self.get_cuda_device_or_cpu()
        except:
            device = "cpu"
        print(f"Now, it is working on {device}.")

        self.model.to(device)
        self.model.train()

        hist = torch.zeros(0)

        for _ in tqdm(range(epochs)):
            tmp = self.run(
                self.model, self.train_dataloader, self.optimizer, device, calc_loss
            )
            hist = torch.cat([hist, tmp])

        return hist